# SMPL-X + FLAME (Fused) Sequence Preview
This notebook loads tracking PKLs, builds the fused SMPL-X model (with FLAME head merging built-in), renders a quick shaded preview, and writes H.264 MP4 output.

In [1]:
# Imports + environment setup (headless-friendly)
import os
# Work from repo root to keep relative imports consistent
os.chdir('../')

import sys
from pathlib import Path
import subprocess
import shutil

import numpy as np
import torch
import cv2
import pickle

# Project rendering utils
from renderer.util import batch_orth_proj, vertex_normals, face_vertices
from renderer.renderer import Renderer

# Make local fused SMPL-X wrapper importable (uses smplx/SMPLX.py)
sys.path.append('/mnt/fasttalk_upperbody/smplx')
from SMPLX import SMPLX as SMPLX_Fused

In [2]:
base_path = "/mnt/GUAVA/assets/example/tracked_video/6gvP8f5WQyo__056/"

## Select Input Sequence
Set `base_path` to the directory containing the tracking PKLs.

In [3]:
tracking_path = base_path + "optim_tracking_ehm.pkl"

with open(tracking_path, "rb") as f:
    data = pickle.load(f)

# Print the keys of the dictionary to see what it contains
print(data.keys())
print(data['frame_000000'].keys())

dict_keys(['frame_000000', 'frame_000001', 'frame_000002', 'frame_000003', 'frame_000004', 'frame_000005', 'frame_000006', 'frame_000007', 'frame_000008', 'frame_000009', 'frame_000010', 'frame_000011', 'frame_000012', 'frame_000013', 'frame_000014', 'frame_000015', 'frame_000016', 'frame_000017', 'frame_000018', 'frame_000019', 'frame_000020', 'frame_000021', 'frame_000022', 'frame_000023', 'frame_000024', 'frame_000025', 'frame_000026', 'frame_000027', 'frame_000028', 'frame_000029', 'frame_000030', 'frame_000031', 'frame_000032', 'frame_000033', 'frame_000034', 'frame_000035', 'frame_000036', 'frame_000037', 'frame_000038', 'frame_000039', 'frame_000040', 'frame_000041', 'frame_000042', 'frame_000043', 'frame_000044', 'frame_000045', 'frame_000046', 'frame_000047', 'frame_000048', 'frame_000049', 'frame_000050', 'frame_000051', 'frame_000052', 'frame_000053', 'frame_000054', 'frame_000055', 'frame_000056', 'frame_000057', 'frame_000058', 'frame_000059', 'frame_000060', 'frame_000061

## Inspect Tracking PKL
Quickly peek at the top-level keys and the first frame’s fields.

In [4]:
# Check what's actually in the nested structure
sample_frame = data['frame_000000']
print("Keys in frame:", sample_frame.keys())
print("\nKeys in smplx_coeffs:", sample_frame['smplx_coeffs'].keys())
print("\n=== Detailed structure of smplx_coeffs ===")
for key, value in sample_frame['smplx_coeffs'].items():
    if isinstance(value, np.ndarray):
        print(f"{key}: shape={value.shape}, dtype={value.dtype}, first few values: {value.flatten()[:5]}")
    else:
        print(f"{key}: type={type(value)} -> {value}")

Keys in frame: dict_keys(['body_crop', 'dwpose_raw', 'dwpose_rlt', 'smplx_coeffs', 'head_crop', 'head_lmk_203', 'head_lmk_70', 'head_lmk_mp', 'flame_coeffs', 'left_mano_coeffs', 'left_hand_crop', 'right_mano_coeffs', 'right_hand_crop'])

Keys in smplx_coeffs: dict_keys(['exp', 'global_pose', 'body_pose', 'body_cam', 'camera_RT_params', 'left_hand_pose', 'right_hand_pose'])

=== Detailed structure of smplx_coeffs ===
exp: shape=(50,), dtype=float32, first few values: [1.2100561  0.5304717  0.11870743 0.19686127 0.08652326]
global_pose: shape=(3,), dtype=float32, first few values: [ 2.9868624   0.06696548 -0.25063005]
body_pose: shape=(21, 3), dtype=float32, first few values: [-0.05537486  0.08022741  0.00862682 -0.0110795  -0.00271257]
body_cam: shape=(3,), dtype=float32, first few values: [1.9841318  0.05343538 0.83356845]
camera_RT_params: shape=(3, 4), dtype=float32, first few values: [-0.9999938  -0.00333516  0.00106661 -0.04895249  0.00326747]
left_hand_pose: shape=(15, 3), dtype=f

## Inspect `smplx_coeffs` Structure
Print shapes and a few values to confirm expected dimensions and ranges.

In [5]:
# Build fused SMPL-X model, parse coeffs, and prep camera/render context (no PCA hands, keep 45D)
from pathlib import Path

# Files
shape_path = base_path + 'id_share_params.pkl'
tracking_path = base_path + 'optim_tracking_ehm.pkl'

# Load shape data if available, otherwise use defaults
if Path(shape_path).exists():
    with open(shape_path, 'rb') as f:
        shape_data = pickle.load(f)
    betas_np = np.asarray(shape_data.get('smplx_shape'))  # often (1, 200)
    betas_np = betas_np.reshape(1, -1) if betas_np.ndim == 1 else betas_np
    print('Loaded betas from file')
else:
    # Use default neutral shape (zeros)
    betas_np = np.zeros((1, 10), dtype=np.float32)
    print('Shape file not found, using default neutral betas')

betas_full = torch.from_numpy(betas_np).float()
print('Betas provided:', betas_full.shape[1])

# Load tracking data
with open(tracking_path, 'rb') as f:
    tracking = pickle.load(f)

frame_keys = sorted([k for k in tracking.keys() if k.startswith('frame_')])
assert len(frame_keys) > 0, 'No frames found in tracking PKL'
print('Frames:', len(frame_keys))

# Inspect first frame to determine expression dim
def get_inner_coeffs(fd):
    return fd['smplx_coeffs'] if isinstance(fd, dict) and 'smplx_coeffs' in fd else fd

sample = get_inner_coeffs(tracking[frame_keys[0]])

def get_len(d, keys, default=None):
    for k in keys:
        if k in d and d[k] is not None:
            v = np.asarray(d[k]).reshape(-1)
            return v.shape[0]
    return default

expr_dim_in = get_len(sample, ['expression','expr'], 50)
body_dim_in = get_len(sample, ['body_pose','body','pose_body'], 63)
print(f'Found dims -> expr:{expr_dim_in} body:{body_dim_in}')

# Use the provided fused SMPLX wrapper that handles head/hand regions internally
from SMPLX import SMPLX as SMPLX_Fused
smplx_model_dir = '/mnt/fasttalk_upperbody/smplx'  # assets dir with SMPLX npz and aux files
assert Path(smplx_model_dir).exists(), f'Missing SMPLX assets at {smplx_model_dir}'
smplx_fused = SMPLX_Fused(smplx_model_dir, n_shape=200, n_exp=50).to('cuda' if torch.cuda.is_available() else 'cpu').eval()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Expected dims for parsing (match fused model)
n_exp_model = 50

# Faces (triangles) for rendering: use fused model's faces directly
faces_t = smplx_fused.faces_tensor[None, ...].to(device)

def to_row_tensor(arr):
    if arr is None:
        return None
    t = torch.from_numpy(np.asarray(arr).reshape(-1)).float()
    return t[None, :]

def fit_dim_row(t, target):
    if t is None:
        return torch.zeros(1, target).float()
    if t.shape[1] > target:
        return t[:, :target]
    if t.shape[1] < target:
        return torch.cat([t, torch.zeros(1, target - t.shape[1], dtype=t.dtype)], dim=1)
    return t

def first_val(d, keys):
    for k in keys:
        if k in d and d[k] is not None:
            return d[k]
    return None

def parse_frame(fd):
    # Align with project data_loader: use only smplx_coeffs entries, no MANO fallbacks
    d = get_inner_coeffs(fd)

    # Global/body/face
    go  = to_row_tensor(first_val(d, ['global_pose','global_orient','root_orient','orient']))
    bp  = to_row_tensor(first_val(d, ['body_pose','body','pose_body']))
    jp  = to_row_tensor(first_val(d, ['jaw_pose','jaw']))
    lep = to_row_tensor(first_val(d, ['leye_pose','left_eye_pose']))
    rep = to_row_tensor(first_val(d, ['reye_pose','right_eye_pose']))

    # Hands: strictly use SMPL-X hand pose from smplx_coeffs (axis-angle, 45D)
    lhp = to_row_tensor(d.get('left_hand_pose'))
    rhp = to_row_tensor(d.get('right_hand_pose'))

    exp = to_row_tensor(first_val(d, ['exp','expression','expr']))
    trn = to_row_tensor(first_val(d, ['transl','translation','trans']))

    # Fit dims (keep as flat row tensors, SMPL-X expects this)
    go  = fit_dim_row(go, 3)                         # (1,3)
    bp  = fit_dim_row(bp, 63)                        # (1,63)
    jp  = fit_dim_row(jp, 3)                         # (1,3)
    lep = fit_dim_row(lep, 3)                        # (1,3)
    rep = fit_dim_row(rep, 3)                        # (1,3)
    lhp = fit_dim_row(lhp, 45)                       # (1,45)
    rhp = fit_dim_row(rhp, 45)                       # (1,45)
    exp = fit_dim_row(exp, n_exp_model)              # (1,n_exp)
    trn = fit_dim_row(trn, 3)                        # (1,3)
    return {
        'global_orient': go,
        'body_pose': bp,
        'jaw_pose': jp,
        'leye_pose': lep,
        'reye_pose': rep,
        'left_hand_pose': lhp,
        'right_hand_pose': rhp,
        'expression': exp,
        'transl': trn,
    }

# Helper: build param_dict for SMPLX_Fused (axis-angle grouped per joint)
def build_param_dict(coeffs, prefer_flame=True, fd=None):
    # choose expression/eyes from flame if available
    exp = None
    eye_l = coeffs['leye_pose']
    eye_r = coeffs['reye_pose']
    if prefer_flame and isinstance(fd, dict) and 'flame_coeffs' in fd:
        f = fd['flame_coeffs']
        if 'expression' in f or 'exp' in f or 'expression_params' in f:
            v = first_val(f, ['expression','exp','expression_params'])
            if v is not None:
                exp = torch.from_numpy(np.asarray(v).reshape(1, -1)).float()
        le = first_val(f, ['leye_pose','left_eye_pose'])
        re = first_val(f, ['reye_pose','right_eye_pose'])
        if le is not None:
            eye_l = torch.from_numpy(np.asarray(le).reshape(1, -1)).float()
        if re is not None:
            eye_r = torch.from_numpy(np.asarray(re).reshape(1, -1)).float()
    if exp is None:
        exp = coeffs['expression']
    # reshape to [B,*,3]
    B = 1
    param = {
        'shape': betas_full[:, :200].to(device),
        'exp': torch.cat([exp, torch.zeros(B, max(0, 50-exp.shape[1]))], dim=1)[:, :50].to(device),
        'global_pose': coeffs['global_orient'].reshape(B, 1, 3).to(device),
        'body_pose': coeffs['body_pose'].reshape(B, 21, 3).to(device),
        'jaw_pose': coeffs['jaw_pose'].reshape(B, 1, 3).to(device),
        'left_hand_pose': coeffs['left_hand_pose'].reshape(B, 15, 3).to(device),
        'right_hand_pose': coeffs['right_hand_pose'].reshape(B, 15, 3).to(device),
        'eye_pose': torch.cat([eye_l.reshape(B,1,3), eye_r.reshape(B,1,3)], dim=1).to(device),
        # translate whole body by transl via joints_offset
        'joints_offset': coeffs['transl'].reshape(B,1,3).expand(B, smplx_fused.J_regressor.shape[0], 3).to(device),
    }
    return param

Shape file not found, using default neutral betas
Betas provided: 10
Frames: 425
Found dims -> expr:50 body:63


/root/miniconda3/envs/fasttalk/lib/python3.11/site-packages/pytorch3d/ops/laplacian_matrices.py:51: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:654.)
  A = torch.sparse.FloatTensor(idx, ones, (V, V))


## Fused SMPL-X Setup + Parsers
Build the fused model (which already merges the FLAME head), define simple helpers to parse and pad inputs, and create a param builder that prefers FLAME expression/eyes when present.

In [6]:
# Debug: print fused model internal dimensions (no vanilla smplx)
print('=== Fused SMPL-X model info ===')
print(f'shapedirs: {tuple(smplx_fused.shapedirs.shape)}')
print(f'J_regressor: {tuple(smplx_fused.J_regressor.shape)}')
print(f'faces: {int(smplx_fused.faces_tensor.shape[0])} tris')
print(f'n_shape: {smplx_fused.n_shape}  n_exp: {smplx_fused.n_exp}')
print('='*40)

=== Fused SMPL-X model info ===
shapedirs: (10475, 3, 250)
J_regressor: (55, 10475)
faces: 20908 tris
n_shape: 200  n_exp: 50


## Model Internals
Confirm model buffers and dimensions to help spot mismatches early.

In [7]:
# Quick shape sanity check using fused forward on first frame
with torch.no_grad():
    coeffs0 = parse_frame(tracking[frame_keys[0]])
    param0 = build_param_dict(coeffs0, prefer_flame=True, fd=tracking[frame_keys[0]])
    out0 = smplx_fused(param0)
    v0 = out0['vertices']
    print('vertices shape:', tuple(v0.shape), 'min/max:', float(v0.min()), float(v0.max()))

vertices shape: (1, 10475, 3) min/max: -1.111258625984192 0.5983074903488159


## Sanity Check Vertices
Run one fused forward pass on the first frame to confirm shapes and reasonable values.

In [8]:
# Sanity check hands after parser simplification
fp = parse_frame(tracking[frame_keys[0]])
print('Left hand pose (len):', fp['left_hand_pose'].shape[1], 'first 6:', fp['left_hand_pose'][0, :6])
print('Right hand pose (len):', fp['right_hand_pose'].shape[1], 'first 6:', fp['right_hand_pose'][0, :6])

Left hand pose (len): 45 first 6: tensor([-0.0851,  0.1201, -1.3283,  0.2095, -0.2069, -0.6650])
Right hand pose (len): 45 first 6: tensor([ 0.0041, -0.3212,  0.8929,  0.4129,  0.2101,  1.1837])


In [9]:
# Render full sequence to MP4 using the in-repo Renderer class.
# Vertices are produced by the fused smplx/SMPLX.py forward (no Kabsch, no manual fusion).

class SMPLXRenderer:
    """
    Minimal rasterization wrapper around the project's Renderer utilities.

    - Projects vertices with orthographic projection (`batch_orth_proj`).
    - Adds five front-facing directional lights for simple shading.
    - Computes per-vertex normals and per-face attributes for shading.
    - Returns shaded RGB images in CHW format.

    Parameters
    ----------
    image_size : int
        Output resolution (square).
    """
    def __init__(self, image_size=800):
        # Keep references to static methods on Renderer (already imported in Cell 1)
        self.rasterize_fn = Renderer.rasterize
        self.add_directionlight_fn = Renderer.add_directionlight
        self.image_size = image_size

    def render_smplx(self, vertices, cam_params, faces, device):
        """
        Render a batch of SMPL-X meshes using orthographic projection + simple shading.

        Parameters
        ----------
        vertices : torch.Tensor
            Tensor of shape [B, V, 3] in world space.
        cam_params : torch.Tensor
            Tensor of shape [B, 3] for orthographic scale/translation (s, tx, ty).
        faces : torch.LongTensor
            Tensor of shape [1, F, 3] or [B, F, 3] with triangle indices.
        device : str or torch.device
            Target device for lighting tensors.

        Returns
        -------
        torch.Tensor
            Shaded RGB images as a tensor of shape [B, 3, H, W].
        """
        batch_size = vertices.shape[0]

        # Orthographic projection with small Z push to ensure front visibility
        transformed_vertices = batch_orth_proj(vertices, cam_params)
        transformed_vertices = transformed_vertices.clone()
        transformed_vertices[:, :, 2] = transformed_vertices[:, :, 2] + 10

        # Five simple front lights (xyz + intensity)
        light_positions = torch.tensor([
            [-1, -1, -1], [1, -1, -1], [-1, +1, -1], [1, +1, -1], [0, 0, -1]
        ])[None, :, :].expand(batch_size, -1, -1).float()
        light_intensities = torch.ones_like(light_positions).float() * 1.7
        lights = torch.cat((light_positions, light_intensities), 2).to(device)

        # Per-vertex and per-face normals
        normals = vertex_normals(vertices, faces)
        face_normals = face_vertices(normals, faces)

        # Simple albedo color (greenish) per vertex -> per face
        colors = torch.tensor([12, 156, 91])[None, None, :].repeat(1, vertices.shape[1], 1).float() / 255.0
        colors = colors.to(device)
        face_colors = face_vertices(colors, faces[0:1] if faces.shape[0] == 1 else faces)
        face_colors = face_colors.expand(batch_size, -1, -1, -1)

        # Rasterize with attributes [rgb(3) | normal(3)]
        attributes = torch.cat([face_colors, face_normals], -1)
        rendering = self.rasterize_fn(self, transformed_vertices, faces, attributes)
        albedo_images = rendering[:, :3, :, :]
        normal_images = rendering[:, 3:6, :, :]

        # Lighting/shading in image space
        shading = self.add_directionlight_fn(self, normal_images.permute(0, 2, 3, 1).reshape([batch_size, -1, 3]), lights)
        shading_images = shading.reshape([batch_size, albedo_images.shape[2], albedo_images.shape[3], 3]).permute(0, 3, 1, 2).contiguous()
        shaded_images = albedo_images * shading_images
        return shaded_images

# Encoder and render loop
renderer = SMPLXRenderer(image_size=800)
demo_path = '/mnt/fasttalk_upperbody/demo'
os.makedirs(demo_path, exist_ok=True)
out_video_renderer = os.path.join(demo_path, 'smplx_flame_fused.mp4')
fps = 25
res = 800
use_ffmpeg = shutil.which('ffmpeg') is not None
ffmpeg_proc = None

if use_ffmpeg:
    print('Using ffmpeg (libx264) for H.264 encoding')
    ffmpeg_cmd = [
        'ffmpeg', '-y',
        '-f', 'rawvideo', '-vcodec', 'rawvideo',
        '-pix_fmt', 'rgb24', '-s', f'{res}x{res}',
        '-r', str(fps), '-i', '-',
        '-an', '-vcodec', 'libx264', '-pix_fmt', 'yuv420p',
        '-preset', 'veryfast', '-crf', '18', out_video_renderer
    ]
    ffmpeg_proc = subprocess.Popen(ffmpeg_cmd, stdin=subprocess.PIPE)
else:
    print('ffmpeg not found; falling back to OpenCV (mp4v)')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(out_video_renderer, fourcc, fps, (res, res), True)
    if not writer.isOpened():
        raise RuntimeError('OpenCV VideoWriter init failed')

print(f'Rendering {len(frame_keys)} frames using SMPLX.py fusion...')

# Recompute centering/scale using the fused model first frame
with torch.no_grad():
    coeffs0 = parse_frame(tracking[frame_keys[0]])
    param0 = build_param_dict(coeffs0, prefer_flame=True, fd=tracking[frame_keys[0]])
    out0 = smplx_fused(param0)
    verts0 = out0['vertices']  # [1,V,3]
    center_t = verts0.mean(dim=1, keepdim=True)
    verts0_centered = verts0 - center_t
    scale = float(2.1 / (verts0_centered.abs().max().item() + 1e-6))
    cam_params = torch.tensor([[scale, 0.0, 0.0]], dtype=torch.float32, device=device)

# Main render loop
with torch.no_grad():
    for idx, fk in enumerate(frame_keys):  # slice with [:10] for a quick preview
        coeffs = parse_frame(tracking[fk])
        param = build_param_dict(coeffs, prefer_flame=True, fd=tracking[fk])
        out_fused = smplx_fused(param)
        verts = out_fused['vertices']
        img_t = renderer.render_smplx(verts - center_t, cam_params, faces_t, device)
        img_rgb = (img_t[0].detach().cpu().permute(1, 2, 0).numpy().clip(0, 1) * 255).astype(np.uint8)
        if use_ffmpeg:
            ffmpeg_proc.stdin.write(img_rgb.tobytes())
        else:
            writer.write(img_rgb[:, :, ::-1])
        if (idx + 1) % 5 == 0 or (idx + 1) == len(frame_keys):
            print(f'  Rendered {idx + 1}/{len(frame_keys)} frames')

# Finalize encoder
if use_ffmpeg:
    ffmpeg_proc.stdin.close(); ffmpeg_proc.wait()
else:
    writer.release()
print(f'✓ Saved video to {out_video_renderer}')

Using ffmpeg (libx264) for H.264 encoding
Rendering 425 frames using SMPLX.py fusion...


ffmpeg version 9c33b2f Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/root/miniconda3/envs/fasttalk --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 / 

  Rendered 5/425 frames


  Rendered 10/425 frames


  Rendered 15/425 frames


...

  Rendered 400/425 frames


  Rendered 405/425 frames
  Rendered 410/425 frames


  Rendered 415/425 frames


  Rendered 420/425 frames
  Rendered 425/425 frames
✓ Saved video to /mnt/fasttalk_upperbody/demo/smplx_flame_fused.mp4


frame=  425 fps= 10 q=-1.0 Lsize=    1385kB time=00:00:16.88 bitrate= 672.0kbits/s speed=0.416x    
video:1379kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.421376%
[libx264 @ 0x55ba8b0ff1c0] frame I:2     Avg QP: 6.37  size: 13507
[libx264 @ 0x55ba8b0ff1c0] frame P:107   Avg QP:16.17  size:  6095
[libx264 @ 0x55ba8b0ff1c0] frame B:316   Avg QP:19.46  size:  2317
[libx264 @ 0x55ba8b0ff1c0] consecutive B-frames:  0.7%  0.5%  0.0% 98.8%
[libx264 @ 0x55ba8b0ff1c0] mb I  I16..4: 75.8% 12.1% 12.1%
[libx264 @ 0x55ba8b0ff1c0] mb P  I16..4:  3.1%  4.1%  1.9%  P16..4:  6.7%  5.7%  3.1%  0.0%  0.0%    skip:75.4%
[libx264 @ 0x55ba8b0ff1c0] mb B  I16..4:  0.5%  0.2%  0.1%  B16..8:  8.1%  4.3%  0.8%  direct: 2.6%  skip:83.3%  L0:46.2% L1:41.9% BI:11.8%
[libx264 @ 0x55ba8b0ff1c0] 8x8 transform intra:37.1% inter:7.7%
[libx264 @ 0x55ba8b0ff1c0] coded y,uvDC,uvAC intra: 40.0% 60.3% 34.7% inter: 2.3% 4.6% 0.9%
[libx264 @ 0x55ba8b0ff1c0] i16 v,h,dc,p: 74%  8%  6% 11%
[l

## Render to MP4 (H.264)
Render all frames with front lights, orthographic projection, and encode via `ffmpeg` using `libx264` + `yuv420p`.